# Import Statements
---
**Important note:**
For some reason tensorflow version and numpy version have dependency conflicts. Need to figure out what version is stable for both of these to work together.

In [11]:

import pandas as pd
import glob
import os
import time
import twint
import nest_asyncio
import re
import nltk 
from nltk.corpus import stopwords   # supplied list of words that can be removed from tweets because they carry no sentiment.
nest_asyncio.apply()
from textblob import TextBlob

#import tensorflow as tf

## Reading in crypto price dataset
---
Section below reads csv files into pandas dataframes for interacting with. Also compiles list of coin names for twitter searching.

### What to do next:
* Retrieve Token labels from CSV file for searching by Cashtag on twitter.

In [3]:
path = r'c:\Users\WaKaBurd\Documents\GitHub\CryptoPredictionTool\archive'
extension = 'csv'
os.chdir(path)
csv_files = glob.glob('*.{}'.format(extension))

# Compile list of all coin names for searching on twitter later
coins = []

for coin in csv_files:
    vals = coin.split("_")
    coin_name = vals[1][:-4]
    coins.append(coin_name)

# compile list of pandas dataframe for assessment
coin_data = []

for file in csv_files:
    df = pd.read_csv(file)
    coin_data.append(df)

## Scrape Twitter for data on all coins supplied by dataset
---
Below section of code searches through twitter using keywords. Uses sift_tweet() function to remove all unnecessary characters, links, emojis & words from tweets.

### What to do next:
* Search twitter based on Cashtags & Hashtags

In [38]:
# Need to create function for cleaning the tweets.
def sift_tweet(tweet, stop_words):
    cleaned_tweet = tweet
    cleaned_tweet = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet) # regex to remove all @userame, emojis, and links from tweets.
    for word in cleaned_tweet:
        if word in stop_words: cleaned_tweet.replace(word, '')
    return cleaned_tweet

# Function for iterating through coins list and storing findings in .csv files
def search_coins(coins):
    important_cols = ['date', 'created_at', 'tweet']
    stopwords_file = open("../archive/stopwords.txt", "r+")
    stopwords = list(stopwords_file.read().split('\n'))
    path = r'c:\Users\WaKaBurd\Documents\GitHub\CryptoPredictionTool\search_results'
    os.chdir(path)
    search_result = pd.DataFrame()
    
    for coin in coins:
        print('performing twitter search for coin:', coin)
    
        timestr = time.strftime("%Y%m%d")

        c = twint.Config()
        c.Limit = 100
        c.Lang = "en"
        c.Pandas = True
        c.Search = coin
        c.Hide_output = True
        twint.run.Search(c)
        coin_df = twint.storage.panda.Tweets_df
        
        if coin_df['tweet'] is None:
            continue
        
        for col in coin_df:
            if col not in important_cols: coin_df.pop(col)
        
        coin_df['Processed Tweet'] = coin_df['tweet'].apply(lambda x: sift_tweet(x.lower(), stopwords))       # Lambda function for creating processed tweets in Coin Dataframe 
        coin_df['Polarity'] = coin_df['Processed Tweet'].apply(lambda x: TextBlob(x).sentiment[0])            # Lambda function for creating Polarity value in Coin Dataframe using Textblob
        
        coin_df.to_csv(coin + '_result.csv')
    print("coin_df columns: ", coin_df.columns)
          
search_coins(coins)

performing twitter search for coin: Aave
performing twitter search for coin: BinanceCoin


KeyboardInterrupt: 

## Perform sentiment analysis using Textblob on cleaned tweets.
---
Section will take cleaned tweets from above and analyze their sentiment for prediction model.

### Dependent on: 
* need tweets to be cleaned before I can complete this section.


### What to do for this section:
* See article: https://towardsdatascience.com/my-absolute-go-to-for-sentiment-analysis-textblob-3ac3a11d524
* Modify above implementation for my own needs.
* Likely need to store a column in the dataframe tracking the polarity of the tweet's sentiment.